In [2]:
import torch

# Parameters
HIDDEN_SIZE = 100
BATCH_SIZE = 256
N_LAYER = 2
N_EPOCHS = 100
N_CHARS = 128
USE_GPU = False

In [ ]:
class NameDataset(Dataset):
    def __init__(self, is_train_set=True):
        filename = 'data/names_train.csv.gz' if is_train_set else 'data/names_test.csv.gz'
        with gzip.open(filename, 'rt') as f:
            reader = csv.reader(f)
            rows = list(reader)  # 数据结构为 [name,country]
        self.names = [row[0] for row in rows] # 得到姓名列表[name1,name2,name3...]
        self.len = len(self.names) # 姓名/数据集总数
        self.countries = [row[1] for row in rows] # 得到国家列表
        self.country_list = list(sorted(set(self.countries))) # 国家列表去重再降序排序
        self.country_dict = self.getCountryDict() # 转化为字典 country:id
        self.country_num = len(self.country_list) # 国家总数
        
    def __getitem__(self, index):
        return self.names[index], self.country_dict[self.countries[index]]
    def __len__(self):
        return self.len

    def getCountryDict(self):
        country_dict = dict()
        for idx, country_name in enumerate(self.country_list, 0):
            country_dict[country_name] = idx
        return country_dict
    def idx2country(self, index): # 基于索引返回国家
        return self.country_list[index]
    def getCountriesNum(self):
        return self.country_num


In [ ]:
trainset = NameDataset(is_train_set=True)
trainloader = DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True)
testset = NameDataset(is_train_set=False)
testloader = DataLoader(testset, batch_size=BATCH_SIZE, shuffle=False)

N_COUNTRY = trainset.getCountriesNum() #获得国家数，也就是类别数，也是最后模型输出的维度

In [ ]:
class RNNClassifier(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size, n_layers=1, bidirectional=True):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size #这里即输出维度
        self.n_layers = n_layers # 网络层数
        self.n_directions = 2 if bidirectional else 1 # 是否双向
        self.embedding = torch.nn.Embedding(input_size, hidden_size) # inputsize为，hiddensize同上，为输出的维度，这里是把希望的嵌入向量的维度（即输入维度）和输出维度设置为一样了
        self.gru = torch.nn.GRU(hidden_size, hidden_size, n_layers,
                bidirectional=bidirectional) # 因为嵌入向量的维度（即输入维度）和输出维度设置为一样，所以两个都用hiddensize了
        self.fc = torch.nn.Linear(hidden_size * self.n_directions, output_size) # outputsize应该和N_COUNTRY是一致的
    def _init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers * self.n_directions,
                batch_size, self.hidden_size)
        return create_tensor(hidden)

    def forward(self, input, seq_lengths):
        # input shape : B x S -> S x B
        input = input.t() # 输入前自己有数是B x S还是 S x B吧
        batch_size = input.size(1) # 取batchsize，仅仅用于初始化隐藏层
        
        hidden = self._init_hidden(batch_size)
        embedding = self.embedding(input) #得到初步输入 seq * batchsize * hiddensize
       
        # pack them up
        gru_input = pack_padded_sequence(embedding, seq_lengths) #一个特殊的打包处理，反正是有利于数据计算加速的
        # 这里的embedding来自于input的转化，而input是在该类之外进行处理的（完成了0填充补全）
        # seq_lengths同样也是在外面处理好再输入的，以张量的形式按顺序存储input的每个batch的长度
        
        output, hidden = self.gru(gru_input, hidden) # 这里的gru_input不用管太多，pack_padded_sequence会自动处理成RNN类能计算的格式
        if self.n_directions == 2:
            hidden_cat = torch.cat([hidden[-1], hidden[-2]], dim=1)
        else:
            hidden_cat = hidden[-1]
        fc_output = self.fc(hidden_cat)
        return fc_output



In [ ]:
# 制作输入数据，这个才是最麻烦的
def make_tensors(names, countries): #names和countries为list形式的数据和标签索引
    sequences_and_lengths = [name2list(name) for name in names]
    # name2list 首先将每个名字里的一个个字母提取出来变为字母列表，并且还返回列表长度，整体数据为
    # [(['a','b',....],3),([],len),([],len)...] 大概长这样

    name_sequences = [sl[0] for sl in sequences_and_lengths] #sl为([],len)
    seq_lengths = torch.LongTensor([sl[1] for sl in sequences_and_lengths])
    countries = countries.long() #转变为longtensor
    # name_sequences——[['a','b','c'],[],[]...] 大概长这样
    # seq_lengths——[1,2,3,4,5...]


    # make tensor of name, BatchSize x SeqLen
    seq_tensor = torch.zeros(len(name_sequences), seq_lengths.max()).long() # 得到全为0的batch*seq的longtensor，注意这里的seq是最长的那个名字，完成了用0补全
    for idx, (seq, seq_len) in enumerate(zip(name_sequences, seq_lengths), 0):
        seq_tensor[idx, :seq_len] = torch.LongTensor(seq) # 用真实数据一个个覆盖全0的数据
    # sort by length to use pack_padded_sequence
    seq_lengths, perm_idx = seq_lengths.sort(dim=0, descending=True) #训练数据长度重排
    seq_tensor = seq_tensor[perm_idx] #根据重排后的索引顺序调整训练数据顺序
    countries = countries[perm_idx] #根据重排后的索引顺序调整标签顺序
    return create_tensor(seq_tensor), \
        create_tensor(seq_lengths),\
        create_tensor(countries)

    """
    def create_tensor(tensor):
    if USE_GPU:
    device = torch.device("cuda:0")
    tensor = tensor.to(device)
    return tensor
    """

In [7]:
def trainModel():
    total_loss = 0
    for i, (names, countries) in enumerate(trainloader, 1):
        inputs, seq_lengths, target = make_tensors(names, countries)
        output = classifier(inputs, seq_lengths)
        loss = criterion(output, target)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        if i % 10 == 0:
            print(f'[{time_since(start)}] Epoch {epoch} ', end='')
            print(f'[{i * len(inputs)}/{len(trainset)}] ', end='')
            print(f'loss={total_loss / (i * len(inputs))}')
    return total_loss


In [ ]:
def testModel():
    correct = 0
    total = len(testset)
    print("evaluating trained model ...")
    with torch.no_grad():
        for i, (names, countries) in enumerate(testloader, 1):
            inputs, seq_lengths, target = make_tensors(names, countries)
            output = classifier(inputs, seq_lengths)
            pred = output.max(dim=1, keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
        percent = '%.2f' % (100 * correct / total)
        print(f'Test set: Accuracy {correct}/{total} {percent}%')
    return correct / total

"""
output.max(dim=1, keepdim=True)：

dim=1：表示沿着 列（类别维度）进行排序，即对于每一行（每个样本），找到最大的预测值。
keepdim=True：表示保持原有的维度。默认情况下，max 操作会将某个维度压缩掉（即减少一个维度）。如果 keepdim=True，则输出的维度保持一致，只不过会将该维度的大小变为 1。这样能保留输出形状，避免后续的形状错误。
结果会返回一个包含两个元素的元组：
第一个元素是 每行的最大值（即最大得分），形状为 (batch_size, 1)。
第二个元素是 最大值的索引，即每个样本的预测类别索引，形状为 (batch_size, 1)。
[1]：我们需要的是 最大值的索引，而不是最大得分，所以通过 [1] 来提取第二个元素，即 预测的类别索引。所以 pred 的形状为 (batch_size, 1)，表示每个样本的预测类别。

pred.eq(target.view_as(pred))：
pred 是模型的预测类别索引，形状为 (batch_size, 1)。
target 是实际的标签，通常是一个张量，表示每个样本的真实类别。其形状也是 (batch_size, 1)。
target.view_as(pred)：将 target 的形状调整为与 pred 相同的形状。这里 view_as(pred) 只是确保 target 和 pred 的形状一致，避免维度不匹配。
pred.eq(target.view_as(pred))：这是一个元素级的比较操作，会返回一个布尔张量，其中每个元素是 True 或 False，表示每个样本的预测是否与真实标签匹配。结果的形状为 (batch_size, 1)。
"""

In [ ]:
if __name__ == '__main__':
    classifier = RNNClassifier(N_CHARS, HIDDEN_SIZE, N_COUNTRY, N_LAYER)
    if USE_GPU:
        device = torch.device("cuda:0")
        classifier.to(device)
    
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(classifier.parameters(), lr=0.001)
    
    # start = time.time() 
    print("Training for %d epochs..." % N_EPOCHS)
    acc_list = []
    for epoch in range(1, N_EPOCHS + 1):
        # Train cycle
        trainModel()
        acc = testModel()
        # acc_list.append(acc)